In [1]:
import arkouda as ak
import arachne as ar
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import os
import pandas as pd

# Connect to Arkouda server
ak.connect("n115", 5555)



    _         _                   _       
   / \   _ __| | _____  _   _  __| | __ _ 
  / _ \ | '__| |/ / _ \| | | |/ _` |/ _` |
 / ___ \| |  |   < (_) | |_| | (_| | (_| |
/_/   \_\_|  |_|\_\___/ \__,_|\__,_|\__,_|
                                          

Client Version: v2024.06.21


connected to arkouda server tcp://*:5555


In [2]:

cluster_dict = {}

print("reading the network file...")
# Read the TSV file using pandas
network_df = pd.read_csv("/scratch/users/md724/arkouda-njit/arachne/data/wcc/test_network_simple_1.tsv", sep=r'\s+', header=None, names=["src", "dst"])
#network_df['type'] = 'T1'
# Create an undirected graph from the data
G = nx.from_pandas_edgelist(network_df, source='src', target='dst', create_using=nx.Graph())

# Print the number of nodes and edges in the graph
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
print(f"Graph has {num_nodes} nodes and {num_edges} edges.")


# Create an Arachne Graph.
ar_network_graph = ar.Graph()
ar_network_graph.add_edges_from(ak.array(network_df["src"]), ak.array(network_df["dst"]))

filePath = "/scratch/users/md724/arkouda-njit/arachne/data/wcc/test_clustering_simple_1.tsv"
print("Running Arachne...") 



reading the network file...
Graph has 20 nodes and 23 edges.
Running Arachne...


In [3]:
clusters = ar.well_connected_components(ar_network_graph, filePath, "/scratch/users/md724/DataSets/UIUC/wccOutPut")
# Function to extract information
def extract_cluster_info(cluster_array):
    cluster_info = []
    for i in range(0, len(cluster_array), 3):
        cluster_id = cluster_array[i]
        depth = cluster_array[i+1]
        members = cluster_array[i+2]
        cluster_info.append({
            'cluster_id': cluster_id,
            'depth': depth,
            'members': members
        })
    return cluster_info

# Extract information
info = extract_cluster_info(clusters)
print("and it returned these WCC Clusters:")
# Print the extracted information
for cluster in info:
    print(f"Cluster ID: {cluster['cluster_id']}, Depth: {cluster['depth']}, Members: {cluster['members']}")
#print("clusters = ", clusters)
#print("clusters.size = ", clusters.size)
ak.shutdown()

Cluster files written to:  /scratch/users/md724/DataSets/UIUC/wccOutPut
and it returned these WCC Clusters:
Cluster ID: 200, Depth: 0, Members: 4
Cluster ID: 100, Depth: 0, Members: 3
